<a href="https://colab.research.google.com/github/JollygreenG-10/Prediction-of-House-Prices/blob/main/Housing_Prices_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 1: Prediction of Housing Prices

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from time import time
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from google.colab import files
uploaded = files.upload() 

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train['logSales'] = np.log(train['SalePrice'])

features = train.drop(['SalePrice', 'logSales'], axis = 1)
labels = train['logSales']
features['train_code'] = 1
test['train_code'] = 0

dfs = [features, test]
opSet = pd.concat(dfs)
print(opSet.head())

''' print(opSet.info()) '''


def counter(col_name):
  return (train[col_name].isna().sum())
for i in train.columns:
    na_counter = counter(i)
    if na_counter > 0:
      print(f"the column {i} has {na_counter} NA values")

def clean_data(df):
  replace_na_columns = ["PoolQC", "Fence", "MiscFeature", 'LotFrontage', 'MasVnrArea']
  for k in replace_na_columns:
    df[k] = df[k].fillna(0)
  meangrgYr = np.mean(df["GarageYrBlt"])
  df["GarageYrBlt"] = df["GarageYrBlt"].fillna(meangrgYr)
  df = pd.get_dummies(df, dummy_na = True)
  return(df)

cleaned = clean_data(opSet)
''' print(cleaned.info()) '''



   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  train_code  
0   2008        WD  

' print(cleaned.info()) '

In [ ]:
tester = cleaned.query("train_code == 0")
trainer = cleaned.query('train_code == 1')
print(trainer.head())
trainer = trainer.drop(["Id"], axis =1)
x_train, x_test, y_train, y_test = train_test_split(trainer, labels, test_size = 0.1, random_state = 29)






   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleType_Oth  SaleType_WD  \
0          2003       196.0       706.0  ...             0            1   
1          1976         0.0       978.0  ...             0            1   
2          2002       162.0       486.0  ...             0            1   
3          1970         0.0       216.0  ...             0            1   
4          2000       350.0       655.0  ...             0            1   

   SaleType_nan  SaleCondition_Abnorml  SaleCondition_AdjLand  \
0  

In [ ]:
estimator = LinearRegression()
feat_select = RFECV(estimator, step = 1)
feat_select = feat_select.fit(x_train, y_train)



In [ ]:
for col in tester.columns:
  tester[col] = tester[col].fillna(0)

def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    end = time()
    model_r2 = r2_score(labels, pred)
    mse = mean_squared_error(labels, pred)
    mae = mean_absolute_error(labels, pred)
    print('{} -- model R^2: {}, MSE: {}, MAE: {}, latency: {}ms'. format(name, model_r2, mse, mae, round((end-start)*1000, 1)))

def counter(col_name):
  return (tester[col_name].isna().sum())
for i in tester.columns:
    na_counter = counter(i)
    if na_counter > 0:
      print(f"the column {i} has {na_counter} NA values")
evaluate_model('LR1', feat_select, x_test, y_test)

tester2 = tester.drop(['Id'], axis = 1)
tester2['preds'] = np.exp(feat_select.predict(tester2))
submissiondf = pd.DataFrame({'Id': tester['Id'], 'SalePrice': tester2['preds']})


LR1 -- model R^2: 0.5831432593237358, MSE: 0.06638239994856737, MAE: 0.10567154855207693, latency: 5.3ms


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(submissiondf.isna().sum())
submissiondf.to_csv('submissiondf.csv', index = False)

Id           0
SalePrice    0
dtype: int64
